In [5]:
# Test case
import torch
from torch_geometric.data import Batch, Data
from torch_geometric.loader import DataLoader

# PairData class
class PairData(Data):
    def __inc__(self, key, value, *args, **kwargs):
        if key == 'edge_index1':
            return self.x1.size(0)
        if key == 'edge_index2':
            return self.x2.size(0)
        return super().__inc__(key, value, *args, **kwargs)


# Create a dummy graph using PyG's Data class
def create_sample_graph(num_nodes, num_node_features, num_edge_features):
    x1, x2 = torch.randn(num_nodes, num_node_features), torch.randn(num_nodes, num_node_features)  # Node features
    edge_index1 = torch.tensor([[0, 2, 3, 1, 4],
                                [2, 0, 1, 3, 3]], dtype=torch.long)
    edge_index2 = edge_index1
    num_edges = edge_index1.size()[1]
    edge_attr1, edge_attr2 = torch.randn((num_edges, num_edge_features)), torch.randn((num_edges, num_edge_features))   # Edge attributes
    
    y = torch.randint(0, 2, (1,))
    y = y.to(torch.float32)
    
    return PairData(x1=x1, edge_index1=edge_index1, edge_attr1=edge_attr1,
                    x2=x2, edge_index2=edge_index2, edge_attr2=edge_attr2,
                    y=y)

# Hyperparameters
num_node_features = 10
num_edge_features = 6
hidden_channels = 20
out_channels = 5

# Create dummy data
data_list = []
for i in range(8):
    data_list.append(create_sample_graph(5, 10, 6))

# Dataloader
loader = DataLoader(data_list, batch_size=4, follow_batch=['x1', 'x2'])

# Initialize the model
model = PairsNNConvGATGNN(num_node_features=num_node_features, num_edge_features=num_edge_features, 
                                   hidden_channels=hidden_channels, out_channels=out_channels)

for batch in loader:
    output = model(batch.x1, batch.edge_index1, batch.edge_attr1, batch.x1_batch,
                         batch.x2, batch.edge_index2, batch.edge_attr2, batch.x2_batch)
    print("Output shape:", output.shape)
    print("Output:", output)

Output shape: torch.Size([4])
Output: tensor([0.6102, 0.6110, 0.6044, 0.6141], grad_fn=<SqueezeBackward1>)
Output shape: torch.Size([4])
Output: tensor([0.6032, 0.6014, 0.6148, 0.6091], grad_fn=<SqueezeBackward1>)


In [6]:
# Training test
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCELoss()

for epoch in range(1, 100):
    for batch in loader:
        optimizer.zero_grad()
        out = model(batch.x1, batch.edge_index1, batch.edge_attr1, batch.x1_batch,
                    batch.x2, batch.edge_index2, batch.edge_attr2, batch.x2_batch)
        loss = criterion(out, batch.y.float())
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch}, Loss: {loss.item()}')
    if loss.item() < 1e-5:
        print("Success! Looks like we're overfitting.")
        break

Epoch: 1, Loss: 0.6169528365135193
Epoch: 2, Loss: 0.5765475034713745
Epoch: 3, Loss: 0.5142960548400879
Epoch: 4, Loss: 0.37178027629852295
Epoch: 5, Loss: 0.2732870578765869
Epoch: 6, Loss: 0.1485879123210907
Epoch: 7, Loss: 0.08459926396608353
Epoch: 8, Loss: 0.014959237538278103
Epoch: 9, Loss: 0.0013020497281104326
Epoch: 10, Loss: 0.00015488950884900987
Epoch: 11, Loss: 2.280228363815695e-05
Epoch: 12, Loss: 4.053523753100308e-06
Success! Looks like we're overfitting.


In [2]:
# Load PairData class dataset of graph pairs
import torch
path = r"C:\Users\xmoot\Desktop\Data\ssl-seizure-detection\patient_pseudolabeled\relative_positioning\PyG\jh101_12s_7min_PairData.pt"
data = torch.load(path)

In [3]:
# Setup GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [11]:
# Place into dataloader


In [29]:
# V1: Real training
import time

num_node_features = 1
num_edge_features = 1
hidden_channels = 64
out_channels = 32

model = PairsNNConvGATGNN(num_node_features=num_node_features, num_edge_features=num_edge_features, 
                                   hidden_channels=hidden_channels, out_channels=out_channels).to(device)       
                          
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()

start_time = time.time()

for epoch in range(1, 10):
    i = 0
    for batch in loader:
        optimizer.zero_grad()
        
        # Mode data to GPU
        batch.to(device)

        out = model(batch.x1, batch.edge_index1, batch.edge_attr1, batch.x1_batch,
                    batch.x2, batch.edge_index2, batch.edge_attr2, batch.x2_batch)
        loss = criterion(out, batch.y.float())
        
        # Scales loss, 
        loss.backward()
        optimizer.step()
        i += 1
        print(i)
        if i % 100 == 0:
            elapsed_time = time.time() - start_time
            print(f"Time elapsed for 100 batches: {elapsed_time:.2f} seconds")
            start_time = time.time()
    print(f'Epoch: {epoch}, Loss: {loss.item()}')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
Time elapsed for 100 batches: 95.87 seconds
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
Time elapsed for 100 batches: 6.85 seconds
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
2

KeyboardInterrupt: 

In [5]:
# V2: Real training with autocast and scaler, and BCEWithLogitsLoss() function
import time
from torch.cuda.amp import autocast, GradScaler

# Parameters
num_node_features = 1
num_edge_features = 1
hidden_channels = 64
out_channels = 32
patience = 20

# Initialize scaler
scaler = GradScaler()

# Initialize model and bind to GPU
model = PairsNNConvGATGNN(num_node_features=num_node_features, num_edge_features=num_edge_features, 
                                   hidden_channels=hidden_channels, out_channels=out_channels).to(device)       

# Optimizer and loss              
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()

start_time = time.time()

for epoch in range(1, 10):
    i = 0
    for batch in loader:
        optimizer.zero_grad()
        
        # Mode data to GPU
        batch.to(device)

        with autocast():
            out = model(batch.x1, batch.edge_index1, batch.edge_attr1, batch.x1_batch,
                    batch.x2, batch.edge_index2, batch.edge_attr2, batch.x2_batch, mode="linear")
            loss = criterion(out, batch.y.float().to(device))
        
        # Scales loss, calls backward() on scaled loss to create scaled gradients
        scaler.scale(loss).backward()
        
        # Unscales gradients and calls or skips optimizer.step()
        scaler.step(optimizer)
        
        # Updates the scale for next iteration
        scaler.update()
        
        # Print time elapsed after 100 batches
        i += 1
        if i % 100 == 0:
            elapsed_time = time.time() - start_time
            print(f"Time elapsed for 100 batches: {elapsed_time:.2f} seconds")
            start_time = time.time()
    print(f'Epoch: {epoch}, Loss: {loss.item()}')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
Time elapsed for 100 batches: 52.98 seconds
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
Time elapsed for 100 batches: 5.94 seconds
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
2

KeyboardInterrupt: 

In [13]:
# V3: With validation
import time
import json
from torch.cuda.amp import autocast, GradScaler

# Hyperparameters
num_node_features = 1
num_edge_features = 1
hidden_channels = 64
out_channels = 32
patience = 20  # Number of epochs to wait before early stop if no progress on the validation set

# Dataloaders
train_loader, val_loader = create_data_loaders(data, data_size = 0.05)

# Initialization
scaler = GradScaler()
model = PairsNNConvGATGNN(num_node_features, num_edge_features, hidden_channels, out_channels).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()

best_val_loss = float('inf')
counter = 0  # Counter for early stopping

for epoch in range(1, 100):  # 100 is just an example, set your own number of epochs
    
    #<---------------------Training--------------------->
    model.train()
    start_time = time.time()
    for batch_idx, batch in enumerate(train_loader):
        optimizer.zero_grad()
        batch.to(device)
        
        with autocast():
            out = model(batch.x1, batch.edge_index1, batch.edge_attr1, batch.x1_batch,
                        batch.x2, batch.edge_index2, batch.edge_attr2, batch.x2_batch)
            loss = criterion(out, batch.y.float().to(device))
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Print time elapsed after 100 batches
        if (batch_idx + 1) % 100 == 0:
            elapsed_time = time.time() - start_time
            print(f"Time elapsed for 100 batches: {elapsed_time:.2f} seconds")
            start_time = time.time()

    #<---------------------Validation--------------------->
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            batch.to(device)
            with autocast():
                out = model(batch.x1, batch.edge_index1, batch.edge_attr1, batch.x1_batch,
                            batch.x2, batch.edge_index2, batch.edge_attr2, batch.x2_batch)
                loss = criterion(out, batch.y.float().to(device))
            val_loss += loss.item()

    val_loss /= len(val_loader)  # Average validation loss
    print(f'Epoch: {epoch}, Train Loss: {loss.item()}, Validation Loss: {val_loss}')

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered.")
            break

Time elapsed for 100 batches: 20.30 seconds
Time elapsed for 100 batches: 5.73 seconds
Time elapsed for 100 batches: 5.92 seconds
Time elapsed for 100 batches: 5.73 seconds
Time elapsed for 100 batches: 5.61 seconds
Time elapsed for 100 batches: 5.65 seconds
Time elapsed for 100 batches: 5.73 seconds
Time elapsed for 100 batches: 5.60 seconds
Time elapsed for 100 batches: 5.66 seconds
Time elapsed for 100 batches: 5.61 seconds
Time elapsed for 100 batches: 5.83 seconds
Epoch: 1, Train Loss: 0.7024990320205688, Validation Loss: 0.692836718107092
Time elapsed for 100 batches: 21.70 seconds
Time elapsed for 100 batches: 5.68 seconds


KeyboardInterrupt: 

In [17]:
# V4: With saving
import time
import json
import torch
from torch.cuda.amp import autocast, GradScaler

# Hyperparameters
num_node_features = 1
num_edge_features = 1
hidden_channels = 64
out_channels = 32
epochs = 1
patience = 20  # Number of epochs to wait before early stop


# Dataloaders
train_loader, val_loader = create_data_loaders(data, data_size = 0.05)

# Initialization
scaler = GradScaler()
model = PairsNNConvGATGNN(num_node_features, num_edge_features, hidden_channels, out_channels).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()

best_val_loss = float('inf')
counter = 0  # Counter for early stopping

# Initialize lists to store loss and accuracy data as tuples
train_loss_data = []
val_loss_data = []
train_acc_data = []
val_acc_data = []

for epoch in range(epochs):  # 100 is just an example, set your own number of epochs
    model.train()
    start_time = time.time()
    epoch_train_loss = 0
    correct_train = 0
    total_train = 0

    # Training loop
    for batch_idx, batch in enumerate(train_loader):
        optimizer.zero_grad()
        batch.to(device)
        with autocast():
            out = model(batch.x1, batch.edge_index1, batch.edge_attr1, batch.x1_batch,
                        batch.x2, batch.edge_index2, batch.edge_attr2, batch.x2_batch)
            loss = criterion(out.squeeze(), batch.y.float().to(device))
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        epoch_train_loss += loss.item()
        # Calculate training accuracy
        pred = torch.sigmoid(out)
        pred = (pred > 0.5).float()
        correct_train += (pred.squeeze() == batch.y.float().to(device)).sum().item()
        total_train += len(batch.y)

    epoch_train_loss /= len(train_loader)
    train_accuracy = 100. * correct_train / total_train
    train_loss_data.append((epoch, epoch_train_loss))
    train_acc_data.append((epoch, train_accuracy))

    # Validation loop
    model.eval()
    epoch_val_loss = 0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for batch in val_loader:
            batch.to(device)
            with autocast():
                out = model(batch.x1, batch.edge_index1, batch.edge_attr1, batch.x1_batch,
                            batch.x2, batch.edge_index2, batch.edge_attr2, batch.x2_batch)
                loss = criterion(out.squeeze(), batch.y.float().to(device))
            epoch_val_loss += loss.item()
            # Calculate validation accuracy
            pred = torch.sigmoid(out)
            pred = (pred > 0.5).float()
            correct_val += (pred.squeeze() == batch.y.float().to(device)).sum().item()
            total_val += len(batch.y)

    epoch_val_loss /= len(val_loader)
    val_accuracy = 100. * correct_val / total_val
    val_loss_data.append((epoch, epoch_val_loss))
    val_acc_data.append((epoch, val_accuracy))

    # Print epoch results
    print(f'Epoch: {epoch}, Train Loss: {epoch_train_loss}, Train Accuracy: {train_accuracy}, Validation Loss: {epoch_val_loss}, Validation Accuracy: {val_accuracy}')

    # Early stopping logic
    if epoch_val_loss < best_val_loss:
        best_val_loss = epoch_val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered.")
            break

# Save training and validation data to JSON
with open('train_loss_data.json', 'w') as f:
    json.dump(train_loss_data, f)
with open('val_loss_data.json', 'w') as f:
    json.dump(val_loss_data, f)
with open('train_acc_data.json', 'w') as f:
    json.dump(train_acc_data, f)
with open('val_acc_data.json', 'w') as f:
    json.dump(val_acc_data, f)


Epoch: 0, Train Loss: 0.6931690457465836, Train Accuracy: 53.72852558051727, Validation Loss: 0.6917822110241857, Validation Accuracy: 54.271844660194176


In [18]:
# Testing opening files
import json

# Read the training loss data
with open('train_loss_data.json', 'r') as f:
    train_loss_data = json.load(f)

# Read the validation loss data
with open('val_loss_data.json', 'r') as f:
    val_loss_data = json.load(f)

# Read the training accuracy data
with open('train_acc_data.json', 'r') as f:
    train_acc_data = json.load(f)

# Read the validation accuracy data
with open('val_acc_data.json', 'r') as f:
    val_acc_data = json.load(f)

# Now you can access the data
print(f"Train Loss Data: {train_loss_data}")
print(f"Validation Loss Data: {val_loss_data}")
print(f"Train Accuracy Data: {train_acc_data}")
print(f"Validation Accuracy Data: {val_acc_data}")

Train Loss Data: [[0, 0.6931690457465836]]
Validation Loss Data: [[0, 0.6917822110241857]]
Train Accuracy Data: [[0, 53.72852558051727]]
Validation Accuracy Data: [[0, 54.271844660194176]]
